# Lab 7

## News Classification with Convolutional Neural Networks (CNN)

In this notebook we will use the cnn headline dataset that has 130000 news titles and their category between: Sports, Business, Sci-Tec and World and we will predict the category with a CNN.

Take it slow and notice we will leverage the embeddings we learned before, and notice the care we must make on the shape of the tensors and which are the channels to convolve on what we want (words)

You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

Follow the instructions. Good luck!

In [ ]:
!nvidia-smi

In [ ]:
!pip install textblob 'gensim==4.2.0'

In [ ]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob


TRACE = False
embedding_dim = 300
n_channels = 64
p_dropout = 0.2
epochs=1000
batch_size = 500
corpus_size=100000
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
tokenizer = lambda x: TextBlob(x).words

In [ ]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
fi

if [ ! -f emb_word2vec_format.txt ]; then
    wget -O emb_word2vec_format.txt https://www.dropbox.com/s/cqoacnovxsq1zoe/emb_word2vec_format.txt?dl=0
fi

In [ ]:
!bash get_data.sh

In [ ]:
path = './news.csv'
news = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)

In [ ]:
# We will reuse what we learned in previous labs!
def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [ ]:
news.title.apply(preprocess_text)
news

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)

import gensim.models

sentences = MyCorpus()
word2vec = gensim.models.Word2Vec(sentences=sentences, vector_size=embedding_dim)
word2vec_model = word2vec.wv

In [ ]:
news['label'] = news.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})

# Get the embedding weights and vocab
weights = None
vocab_size = None

In [ ]:
weights.shape

In [ ]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news)

In [ ]:
X = np.zeros((len(news), maximum))   # Here we do what we said above
# Iterate through the news df and for every word, if it exists in the word2vec model, put into X for that review and that word the index of the embedding (check index_to_key)
# FILL
y = news.label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = tf.constant(X_train)
X_test = tf.constant(X_test)
y_train = tf.one_hot(tf.constant(y_train), 4)
y_test = tf.one_hot(tf.constant(y_test), 4)

In [ ]:
X_train[0]

In [ ]:
weights.shape

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=weights.shape[0], output_dim=embedding_dim, input_length=maximum, embeddings_initializer=Constant(weights), trainable=True))
model.add()  # Add a Conv1d layer tp have n_channels filters of 3x3. It is key you ensure you don't loose dimension size!
model.add()  # Add an activation layer
model.add(Conv1D(filters=2*n_channels, kernel_size=3, padding='same', data_format='channels_first')) # Add another Conv1d layer tp have 2*n_channels filters of 3x3. It is key you ensure you don't loose dimension size!
model.add()  # Add an activation layer
model.add()  # Average out the convolution dimension
model.add()  # You can add several Dense and batch norm layers if you prefer
model.add()  # Add a dropout
model.add()  # Final Dense layer

In [ ]:
# Compile the model. Think what is the best loss to use
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01)
history = None # Fit the model, use the callback above to do EarlyStopping

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=2.0)

In [ ]:
# Plot whatever metric you defined in the compilation

In [ ]:
# Evaluate the model
None